> This is a self-correcting activity generated by [nbgrader](https://nbgrader.readthedocs.io). Fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Run subsequent cells to check your code.

---

# Classify handwritten digits

In this activity, you'll try several classifiers on the [UCI handwritten digits dataset](https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits), either separately or into an ensemble.

![UCI digits](images/uci_digits.png)

## Environment setup

In [1]:
# Import base packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
# Setup plots
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8
%config InlineBackend.figure_format = 'retina'
sns.set()

In [3]:
# Import ML packages
import sklearn
print(f'scikit-learn version: {sklearn.__version__}')

from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split

scikit-learn version: 0.23.1


## Step 1: Loading and preparing the data

In [4]:
# Load the MNIST digits dataset
digits = load_digits()

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

print(f'data: {data.shape}. targets: {digits.target.shape}')

df_digits = pd.DataFrame(data)
df_digits.head(n=10)

data: (1797, 64). targets: (1797,)


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0
5,0.0,0.0,12.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,9.0,16.0,16.0,10.0,0.0,0.0
6,0.0,0.0,0.0,12.0,13.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,1.0,9.0,15.0,11.0,3.0,0.0
7,0.0,0.0,7.0,8.0,13.0,16.0,15.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.0,5.0,0.0,0.0,0.0,0.0
8,0.0,0.0,9.0,14.0,8.0,1.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,11.0,16.0,15.0,11.0,1.0,0.0
9,0.0,0.0,11.0,12.0,0.0,0.0,0.0,0.0,0.0,2.0,...,4.0,0.0,0.0,0.0,9.0,12.0,13.0,3.0,0.0,0.0


### Question

Split the data into training, validation and test sets.

In [5]:
x_train_val, x_test_origin, y_train_val, y_test = train_test_split(data, digits.target, test_size=0.2)

x_test = x_test_origin.astype('float32') / 16
x_train_centered = x_train_val.astype('float32') / 16

x_val, x_train = np.split(x_train_centered, [200])
y_val, y_train = np.split(y_train_val, [200])

In [6]:
print(f'x_train: {x_train.shape}. x_val: {x_val.shape}. x_test: {x_test.shape}')
print(f'y_train: {y_train.shape}. y_val: {y_val.shape}. y_test: {y_test.shape}')

assert x_train.shape == (1237, 64)
assert x_val.shape == (200, 64)
assert x_test.shape == (360, 64)
assert y_train.shape == (1237,)
assert y_val.shape == (200,)
assert y_test.shape == (360,)

x_train: (1237, 64). x_val: (200, 64). x_test: (360, 64)
y_train: (1237,). y_val: (200,). y_test: (360,)


## Step 2: Train several models

### Question

Create and train various models, such as a linear classifier, a multilayer perceptron, a random forest...

In [7]:
# Create various models
lr_model = SGDClassifier()
mlp_model = MLPClassifier(max_iter=500)
rf_model = RandomForestClassifier()

# Train them
lr_model.fit(x_train, y_train)
mlp_model.fit(x_train, y_train)
rf_model.fit(x_train, y_train)

RandomForestClassifier()

### Question

Display the default score for each model.

In [8]:
print("Linear Classifier scores :\n")
print(f"Training score {lr_model.score(x_train, y_train) * 100:.2f}%")
print(f"Validation score {lr_model.score(x_val, y_val) * 100:.2f}%")
print(f"Test score {lr_model.score(x_test, y_test) * 100:.2f}%\n")

print("MLP Classifier scores :\n")
print(f"Training score {mlp_model.score(x_train, y_train) * 100:.2f}%")
print(f"Validation score {mlp_model.score(x_val, y_val) * 100:.2f}%")
print(f"Test score {mlp_model.score(x_test, y_test) * 100:.2f}%\n")

print("Random Forest Classifier scores :\n")
print(f"Training score {rf_model.score(x_train, y_train) * 100:.2f}%")
print(f"Validation score {rf_model.score(x_val, y_val) * 100:.2f}%")
print(f"Test score {rf_model.score(x_test, y_test) * 100:.2f}%\n")


Linear Classifier scores :

Training score 98.38%
Validation score 92.50%
Test score 95.56%

MLP Classifier scores :

Training score 100.00%
Validation score 94.50%
Test score 98.33%

Random Forest Classifier scores :

Training score 100.00%
Validation score 96.00%
Test score 96.94%



### Question

Create a `VotingClassifier` including all your models. Fit it on the training data.

In [9]:
# Estimators used by voting classifier, log loss used for soft voting later
estimators = [("linear", SGDClassifier(loss='log')), ("perceptron", mlp_model), ("random forest", rf_model)]
voting_model = VotingClassifier(estimators)

voting_model.fit(x_train, y_train)

VotingClassifier(estimators=[('linear', SGDClassifier(loss='log')),
                             ('perceptron', MLPClassifier(max_iter=500)),
                             ('random forest', RandomForestClassifier())])

### Question

Show the `VotingClassifier` score and compare to each model's individual score.

In [10]:
print("Voting Classifier (hard) score :\n")
print(f"Training score {voting_model.score(x_train, y_train) * 100:.2f}%")

Voting Classifier (hard) score :

Training score 100.00%


### Question

Show the score for a soft voting classifier.

In [11]:
soft_voting_model = VotingClassifier(estimators, voting='soft')

soft_voting_model.fit(x_train, y_train)

print("Voting Classifier (soft) score :\n")
print(f"Training score {soft_voting_model.score(x_train, y_train) * 100:.2f}%")

Voting Classifier (soft) score :

Training score 100.00%


### Question

Compute the `VotingClassifier` score on the test data. Compare it to each model's individual score.

In [12]:
print("Voting Classifier (hard) test score :\n")

print(f"Test score {voting_model.score(x_train, y_train) * 100:.2f}%\n")

print("Voting Classifier (soft) test score :\n")

print(f"Test score {soft_voting_model.score(x_train, y_train) * 100:.2f}%")

Voting Classifier (hard) test score :

Test score 100.00%

Voting Classifier (soft) test score :

Test score 100.00%
